# Capture Face

In [1]:
from tensorflow.keras.models import model_from_json
model = model_from_json(open("detection_model.json", "r").read())
model.load_weights('detection_model.h5')

In [2]:
import cv2
model.load_weights('detection_model.h5')
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
while cap.isOpened():
    if not cap.isOpened():
        break
    ret, frame = cap.read()
    if frame is None:
        break
    height, width, channel = frame.shape

    # convert to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces
    #faces = face_haar_cascade.detectMultiScale(gray_image)
    faces = face_haar_cascade.detectMultiScale(gray_image, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30), maxSize=(300, 300))

   
    sub_img = frame[0:int(height/6), 0:int(width)]
    black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
    res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 0)
    for (x, y, w, h) in faces:
        cv2.rectangle(res, (x, y), (x+w, y+h), (0, 255, 0), 2)
         
        # crop the face
        roi_gray = gray_image[y:y+h, x:x+w]
        #roi_gray = gray_image[y-5:y+h+5, x-5:x+w+5]
        
        # resize the cropped face to 48x48
        roi_gray = cv2.resize(roi_gray, (48, 48))

        # convert image to array
        image_pixels = img_to_array(roi_gray)
        image_pixels = np.expand_dims(image_pixels, axis=0)
        image_pixels /= 255
        
        # predict the emotion
        predictions = model.predict(image_pixels)
        max_index = np.argmax(predictions[0])
        emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion_prediction = emotion_detection[max_index]
        lable = emotion_prediction
        
        # Defining the font properties
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        FONT_SCALE = 0.8
        FONT_THICKNESS = 1

        lable_dimension = cv2.getTextSize(lable, FONT, FONT_SCALE, FONT_THICKNESS)[0]
        textX = int((res.shape[1] - lable_dimension[0]) / 2)
        textY = int((res.shape[0] + lable_dimension[1]) / 2)
        cv2.putText(res, lable, (textX,textY), FONT, FONT_SCALE, (0,0,0), FONT_THICKNESS)
    
    cv2.imshow('Face Emotion Recognition', res)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 39ms/step


In [13]:
#Working perfact
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    if not cap.isOpened():
        break
    ret, frame = cap.read()
    if frame is None:
        break
    height, width, channel = frame.shape

    # resize frame
    frame = cv2.resize(frame, (320, 240))

    # convert to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces
    #faces = face_haar_cascade.detectMultiScale(gray_image, 1.3, 5)
    # detect faces
    faces = face_haar_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    
    if len(faces) > 0:
        # get the largest face
        (x, y, w, h) = max(faces, key=lambda x: x[2] * x[3])

        # draw rectangle on face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # crop the face
        roi_gray = gray_image[y:y+h, x:x+w]

        # resize the cropped face to 48x48
        roi_gray = cv2.resize(roi_gray, (48, 48))

        # convert image to array
        image_pixels = img_to_array(roi_gray)
        image_pixels = np.expand_dims(image_pixels, axis=0)
        image_pixels /= 255

        # predict the emotion
        predictions = model.predict(image_pixels)
        max_index = np.argmax(predictions[0])
        emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        emotion_prediction = emotion_detection[max_index]
        label = emotion_prediction

        # add label to the frame
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        FONT_SCALE = 0.8
        FONT_THICKNESS = 1
        lable_dimension = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)[0]
        textX = int((frame.shape[1] - lable_dimension[0]) / 2)
        textY = int((frame.shape[0] + lable_dimension[1]) / 2)
        cv2.putText(frame, label, (textX,textY), FONT, FONT_SCALE, (0,0,0), FONT_THICKNESS)

    # show the frame
    cv2.imshow('Face Emotion Recognition', frame)

    # wait for user to press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 47ms/step
